<a href="https://colab.research.google.com/github/OrlovAlexeyA/devops_lesson_trial/blob/main/DZ_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 2.1 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 


In [4]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

№1 Агент играющий всегда "Камень":

In [41]:
%%writefile rock.py

def rock(observation, configuration):
    return 0

Overwriting rock.py


№2 Агент "Бумага":

In [88]:
%%writefile paper.py

def your_agenscissors(observation, configuration):
    return 1

Overwriting paper.py


№3 Агент "Ножницы":

In [86]:
%%writefile scissors.py

def scissors(observation, configuration):
    return 2

Overwriting scissors.py


№4 Агент играющий Камень или Бумага, выбирая случайный вариант из них:

In [10]:
%%writefile rock_paper.py

def rock_paper(observation, configuration):
  import random
  return random.choice([0,1])

Writing rock_paper.py


№5 Агент играющий Камень или Ножницы, выбирая случайный вариант из них:

In [17]:
%%writefile rock_scissors.py

def rock_scissors(observation, configuration):
  import random
  return random.choice([0,2])

Writing rock_scissors.py


№6 Агент играющий Бумага или Ножницы, выбирая случайный вариант из них:

In [40]:
%%writefile paper_scissors.py

def paper_scissors(observation, configuration):
  import random
  return random.choice([1,2])

Overwriting paper_scissors.py


№7 Агент, играющий Камень или Бумага или Ножницы, выбирая случайный вариант их них:

In [24]:
%%writefile clear_random.py

def clear_random(observation, configuration):
  import random
  answer = random.choice([0,1,2])
  return answer

Overwriting clear_random.py


№ 8 Агент, отвечающий не так как последний раз ответил оппонент и выбирающий случайный вариант из двух остальных:

In [63]:
%%writefile non_opponent.py


def non_opponent(observation, configuration):
    import random
    lst = [0,1,2]
    if observation.step > 0:
      lst.remove(observation.lastOpponentAction)
      return random.choice(lst)
    else:
        return random.choice(lst)

Overwriting non_opponent.py


№9 Агент, выбирающий вариант, побеждающий последний ответ оппонента

In [81]:
%%writefile win_last_opponent.py


def win_last_opponent(observation, configuration):
    import random
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 1
        if observation.lastOpponentAction == 1:
           return 2
        if observation.lastOpponentAction == 2:
            return 0
    else:
       return random.choice([0,1,2])

Overwriting win_last_opponent.py


Напишем функцию для тестированя игры двух ботов. 100 игр, 5 раундов.

In [103]:
evaluate(
    "rps", 
    ["clear_random.py", "win_last_opponent.py"], 
    configuration={"episodeSteps": 101, "tieRewardThreshold":1}, num_episodes = 5
)

[[-19.0, 19.0], [-1.0, 1.0], [8.0, -8.0], [-6.0, 6.0], [-1.0, 1.0]]

Для автоматизации проведения турнира сделаем список всех Агентов:

In [92]:
agents = ['rock.py', 
          'paper.py',
          'scissors.py',
          'rock_paper.py',
          'rock_scissors.py',
          'paper_scissors.py',
          'clear_random.py',
          'non_opponent.py',
          'win_last_opponent.py']          

Через цикл запустим игру. Каждый агент должен сыграть с каждым оставшимся. Играть будем по 500 игр, 5 раундов чтобы уменьшить влияние отклонения. 

Также создадим список с итоговым результатом для каждого агента. Итоговый рузультат это: "сумма всех побед минус сумма всех поражений". 

Далее у кого больше побед, то и лучший СРЕДИ ДАННОГО СПИСКА АГЕНТОВ НА ДАННОМ РОЗЫГРЫШЕ.

In [120]:
print("Проверка ботов:")
#создадим список результирующих очков
scores = [0 for x in range(9)]
# число раундов
round = 5
for x in range(len(agents)):
  for y in range(x+1, len(agents)):
    result = evaluate(
    "rps", 
    [agents[x], agents[y]], 
    configuration={"episodeSteps": 500, "tieRewardThreshold":1}, num_episodes = round)
    # будем вносить все полученные очки в список scores
    for  z in range(round):
      scores[agents.index(agents[x])] += result[z][0]
      scores[agents.index(agents[y])] += result[z][1]
      
    print(result," ", agents[x],"vs", agents[y]) 
# выведем результирующий список суммарных очков
print("Итоговые результаты: ", scores)
print ("Победитель: ", agents[scores.index(max(scores))], " с ", max(scores))

Проверка ботов:
[[-499.0, 499.0], [-499.0, 499.0], [-499.0, 499.0], [-499.0, 499.0], [-499.0, 499.0]]   rock.py vs paper.py
[[499.0, -499.0], [499.0, -499.0], [499.0, -499.0], [499.0, -499.0], [499.0, -499.0]]   rock.py vs scissors.py
[[-264.0, 264.0], [-275.0, 275.0], [-236.0, 236.0], [-248.0, 248.0], [-256.0, 256.0]]   rock.py vs rock_paper.py
[[260.0, -260.0], [237.0, -237.0], [256.0, -256.0], [251.0, -251.0], [256.0, -256.0]]   rock.py vs rock_scissors.py
[[-7.0, 7.0], [39.0, -39.0], [-7.0, 7.0], [3.0, -3.0], [-1.0, 1.0]]   rock.py vs paper_scissors.py
[[-38.0, 38.0], [16.0, -16.0], [-27.0, 27.0], [28.0, -28.0], [-2.0, 2.0]]   rock.py vs clear_random.py
[[-21.0, 21.0], [76.0, -76.0], [-23.0, 23.0], [13.0, -13.0], [-19.0, 19.0]]   rock.py vs non_opponent.py
[[-498.0, 498.0], [-499.0, 499.0], [-499.0, 499.0], [-498.0, 498.0], [-498.0, 498.0]]   rock.py vs win_last_opponent.py
[[-499.0, 499.0], [-499.0, 499.0], [-499.0, 499.0], [-499.0, 499.0], [-499.0, 499.0]]   paper.py vs scissors.

Уверенно побеждает Агент №9, отвечающий победой на последний ответ оппонента. Видимо это происходит из за того, что играет большое количество "статичных" агентов, у которых он всё время выигрывает и быстро набирает положительную статистику.